In [31]:
import os
import sqlite3
from collections import defaultdict
from datetime import date
import xml.etree.ElementTree as ET


"""
What left to do?
- Send them to google drive
"""

# directory = "/Users/kaanerdem/Desktop/projeler/pdtxt/pdtxt_notebook/Finished Episodes_Word/Keyword Extraction"
directory = "/Users/kaanerdem/Desktop/projeler/pdtxt/pdtxt_notebook/transcripted_episodes/episodes_with_timestamps/new_episodes"
saf_root = "SAF/"

def create_SAF_Folder(episode_number:int):
    # Create the folder
    episode_folder_path = saf_root + str(episode_number)
    print(episode_folder_path)
    os.makedirs(episode_folder_path, exist_ok=True)

    # Connect to cursor to retreive all important information.
    with sqlite3.connect("nasil_olunur_database.db") as connection:
        cursor = connection.cursor()
        query = """SELECT title,episode_date,audio_url,word_document_name,description_tr,description_en
                    FROM episodes WHERE episode_number = ?"""
        cursor.execute(query, (episode_number,))
        episode = cursor.fetchone()
        title,episode_date,audio_url,word_document_name,description_tr,description_en = episode

        # Need to retrieve guest information
        guest_query = """ SELECT guest_name FROM guests WHERE episode_number = ?"""
        cursor.execute(guest_query, (episode_number,))
        guests = cursor.fetchall()
        guests = ' - '.join(name[0] for name in guests) # it's a string now.

        # Need to retrieve subject keywords.
        keyword_query = """SELECT type, keyword_text FROM keywords WHERE episode_number = ?"""
        cursor.execute(keyword_query, (episode_number,))
        keywords = cursor.fetchall()
        kw_dict = defaultdict(list) # key = keyword type, val = keyword text
        for label, kw in keywords:
            kw_dict[label].append(kw)

    # Add the .txt file
    add_txt(episode_folder_path, word_document_name)

    # Create the contents file
    create_contents(episode_folder_path)

    # Create the dublin_core
    # create_dublin_core(episode_number, episode_folder_path)
    create_dublin_core(guests,
                       episode_date,
                       description_tr,
                       description_en,
                       title,
                       audio_url,
                       kw_dict["subject_tr"],
                       kw_dict["subject_en"],
                       episode_number,
                       episode_folder_path)

    # Create the metadata file
    create_metadata(episode_number,
                    episode_folder_path,
                    kw_dict["person"],
                    kw_dict["place_tr"])

# create_SAF_Folder(159)

In [32]:
# Batch Create SAF FOLDERS
def batch_create_SAF(start_ep:int, end_ep:int):

    for i in range(start_ep, end_ep+1):
        try:
            create_SAF_Folder(i)
        except Exception as e:
            print(f"Error when creating saf folder {i}. --> {e}")
    remove_ds_store()
    zip_folders(start_ep,end_ep)

# batch_create_SAF(65,65)
batch_create_SAF(159,234)

SAF/159
SAF/160
SAF/161
SAF/162
SAF/163
SAF/164
SAF/165
SAF/166
SAF/167
SAF/168
SAF/169
SAF/170
SAF/171
SAF/172
SAF/173
SAF/174
SAF/175
SAF/176
SAF/177
SAF/178
SAF/179
SAF/180
SAF/181
SAF/182
SAF/183
SAF/184
SAF/185
SAF/186
SAF/187
SAF/188
SAF/189
SAF/190
SAF/191
SAF/192
SAF/193
SAF/194
SAF/195
SAF/196
SAF/197
SAF/198
SAF/199
SAF/200
SAF/201
SAF/202
SAF/203
SAF/204
SAF/205
SAF/206
SAF/207
SAF/208
SAF/209
SAF/210
SAF/211
SAF/212
SAF/213
SAF/214
SAF/215
SAF/216
SAF/217
SAF/218
SAF/219
SAF/220
SAF/221
SAF/222
SAF/223
SAF/224
SAF/225
SAF/226
SAF/227
SAF/228
SAF/229
SAF/230
SAF/231
Error when creating saf folder 231. --> 'NoneType' object has no attribute 'find'
SAF/232
SAF/233
SAF/234
Zipped: /Users/kaanerdem/Desktop/projeler/pdtxt/pdtxt_notebook/SAF/168 -> /Users/kaanerdem/Desktop/projeler/pdtxt/pdtxt_notebook/SAF/zip/168.zip
Zipped: /Users/kaanerdem/Desktop/projeler/pdtxt/pdtxt_notebook/SAF/159 -> /Users/kaanerdem/Desktop/projeler/pdtxt/pdtxt_notebook/SAF/zip/159.zip
Zipped: /Users/kaane

ValueError: invalid literal for int() with base 10: 'zip'

In [5]:
import shutil
import os

def zip_folders(start_ep:int, end_ep:int):
    directory = "/Users/kaanerdem/Desktop/projeler/pdtxt/pdtxt_notebook/SAF"
    zip_directory = directory + "/zip"

    for item in os.listdir(directory):
        item_path = os.path.join(directory,item)

        #Check if the item is a folder
        if os.path.isdir(item_path) and (start_ep <= int(item) <= end_ep):
            zip_path = os.path.join(zip_directory,item)
            try:
                shutil.make_archive(zip_path, 'zip', item_path)
                print(f"Zipped: {item_path} -> {zip_path}.zip")
            except Exception as e:
                print(f"Problem zipping {item_path}. Error -> {e}")

# zip_folders(61,149)

In [14]:
import os

def remove_ds_store(): # Removes the macOS specific .DS_Store files
    directory = "/Users/kaanerdem/Desktop/projeler/pdtxt/pdtxt_notebook/SAF"
    for root, _, files in os.walk(directory):
        for file in files:
            if file == ".DS_Store":
                os.remove(os.path.join(root, file))

# remove_ds_store()

In [7]:
# Guest querying trial
def deneme_guest_retrieval():
    episode_number = 231
    with sqlite3.connect("nasil_olunur_database.db") as connection:
        cursor = connection.cursor()
        guest_query = """ SELECT guest_name FROM guests WHERE episode_number = ?"""
        cursor.execute(guest_query, (episode_number,))
        guests = cursor.fetchall()
        guests = ' - '.join(name[0] for name in guests)
    return guests
    # return output
# deneme_guest_retrieval()

In [8]:
# Keyword querying trial
def deneme_keyword_retrieval():
    from collections import defaultdict
    episode_number = 61
    with sqlite3.connect("nasil_olunur_database.db") as connection:
        cursor = connection.cursor()
        keyword_query = """
            SELECT type, keyword_text FROM keywords WHERE episode_number = ?
        """
        cursor.execute(keyword_query, (episode_number,))
        keywords = cursor.fetchall()

        kw_dict = defaultdict(list)
        for label, kw in keywords:
            kw_dict[label].append(kw)
            # print(f"Label-> {label}, keyword-> {kw}")

        print(kw_dict.keys())

In [9]:
"""
    Function to create dublin_core file
"""

def create_dublin_core(guests,episode_date, description_tr, description_en, title, audio_url, subject_tr, subject_en, episode_number, episode_folder_path):

    #Clearing audio_url
    audio_url = audio_url[:audio_url.find("mp3")+3]

    dublin_core_metadata_structure = {
        "contributor_author": guests,
        "date_accessioned": str(date.today()),
        "date_issued": str(episode_date), # tire (-) ile ayirmam gerekir mi?
        "episode_no": str(episode_number),
        "identifier_uri": audio_url,
        "description_abstract_en": description_en,
        "description_abstract_tr": description_tr,
        "language_iso": "tr",
        "subjects": {
            "en": subject_en,
            "tr": subject_tr,
        },
        "title": title,
        "type": "Recording, oral",
    }

    # Create the root element
    root = ET.Element("dublin_core", attrib={"schema": "dc"})

    # Add metadata as sub-elements
    ET.SubElement(root, "dcvalue", attrib={"element": "contributor", "qualifier": "author"}).text = dublin_core_metadata_structure["contributor_author"]
    ET.SubElement(root, "dcvalue", attrib={"element": "date", "qualifier": "accessioned"}).text = dublin_core_metadata_structure["date_accessioned"]
    ET.SubElement(root, "dcvalue", attrib={"element": "date", "qualifier": "issued"}).text = dublin_core_metadata_structure["date_issued"]
    ET.SubElement(root, "dcvalue", attrib={"element": "identifier", "qualifier": "none"}).text = dublin_core_metadata_structure["episode_no"]
    ET.SubElement(root, "dcvalue", attrib={"element": "identifier", "qualifier": "uri"}).text = dublin_core_metadata_structure["identifier_uri"]

    # Add descriptions
    ET.SubElement(
        root,
        "dcvalue",
        attrib={"element": "description", "qualifier": "version"}
    ).text = dublin_core_metadata_structure["description_abstract_en"]
    ET.SubElement(
        root,
        "dcvalue",
        attrib={"element": "description", "qualifier": "abstract"}
    ).text = dublin_core_metadata_structure["description_abstract_tr"]

    # Add Language attribute
    ET.SubElement(
        root,
        "dcvalue",
        attrib={"element": "language", "qualifier": "iso"}
    ).text = "tr"

    # Add subjects
    for subject in dublin_core_metadata_structure["subjects"]["en"]:
        ET.SubElement(root, "dcvalue", attrib={"element": "subject", "qualifier": "none"}).text = subject

    for subject in dublin_core_metadata_structure["subjects"]["tr"]:
        ET.SubElement(root, "dcvalue", attrib={"element": "subject", "qualifier": "none"}).text = subject

    # Add title and type
    ET.SubElement(root, "dcvalue", attrib={"element": "title", "qualifier": "none"}).text = dublin_core_metadata_structure["title"]
    ET.SubElement(root, "dcvalue", attrib={"element": "type", "qualifier": "none"}).text = dublin_core_metadata_structure["type"]

    # Write to an XML file
    output_file = os.path.join(episode_folder_path, "dublin_core.xml")
    tree = ET.ElementTree(root)
    tree.write(output_file, encoding="utf-8", xml_declaration=True)

    # print(f"XML file created: {output_file}")

In [10]:
def create_contents(path):
    text = ""
    for item in os.listdir(path):
        if item != ".DS_Store":
            text += item + "\n"

    # save it into a file in correct dir.
    with open(path+"/contents", "w") as f:
        f.write(text)
# create_contents("/Users/kaanerdem/Desktop/projeler/pdtxt/pdtxt_notebook/SAF/61")


In [23]:
from docx import Document

def add_txt(episode_folder_path, word_document_name):
    doc = Document(word_document_name)
    text = "\n".join([para.text for para in doc.paragraphs])

    n_start = word_document_name.find("new_episodes/") + 13
    name = word_document_name[n_start:-5]+".txt".strip()

    with open(f"{episode_folder_path}/{name}", "w") as file:
        file.write(text)


In [29]:
import sqlite3
import os

def add_word_addresses():
    with sqlite3.connect("nasil_olunur_database.db") as connection:
        data = []
        directory = "/Users/kaanerdem/Desktop/projeler/pdtxt/pdtxt_notebook/transcripted_episodes/episodes_with_timestamps/new_episodes"
        for filename in os.listdir(directory):
            if filename.endswith(".docx"):
                ep_no = int(filename[:filename.find("-")].strip())
                full_ad = os.path.join(directory, filename)
                data.append((full_ad,ep_no))

        cursor = connection.cursor()
        query = """
            UPDATE episodes
            SET word_document_name = ?
            WHERE episode_number = ?
        """
        cursor.executemany(query,data)

add_word_addresses()

In [13]:
def create_metadata(episode_number,episode_folder_path, person, place_tr):

    # Create the root element
    root = ET.Element("dublin_core", schema="local")

    # Add people to the XML
    for p in person:
        dcvalue = ET.SubElement(root, "dcvalue")
        dcvalue.set("element", "person")
        dcvalue.set("qualifier", "name")
        dcvalue.text = p

    # Add places to the XML
    for place in place_tr:
        dcvalue = ET.SubElement(root, "dcvalue")
        dcvalue.set("element", "place")
        dcvalue.set("qualifier", "name")
        dcvalue.text = place

    # Create an ElementTree object
    tree = ET.ElementTree(root)

    output_dir = episode_folder_path
    output_file = os.path.join(output_dir, "metadata_local.xml")
    # Write the tree to an XML file
    with open(output_file, "wb") as file:
        file.write(b'<?xml version="1.0" encoding="utf-8" standalone="no"?>\n')
        tree.write(file, encoding="utf-8", xml_declaration=False)
    # print(f"XML file created: {output_file}")